<h3 align = 'center'><font color='#28abe3'>Predicting Red Hat Business Value (Improving on Benchmark)</font></h3>

Update:

After doing a bit of reading of the kernals and forums for this contest there is a big leak in the data which alone produces an AUC of .987. Right now the 10% mark is an AUC of 0.991676 so I definitely need to exploit the leak in order to get into the top 10% in this competiton. The competition mods say there are around 70k test points unaffected by the leak (datapoints with a group_1 class not in the train set) so that last bit of improvement will come from true model improvements, feature engineering, ensembling etc. Lets start with the biggest model improvement first, which is integrating the leak into our model.


---

Import the data:

In [13]:
#import modules
import pandas as pd
import numpy as np
import os 
import sqlite3

#rename columns
act_train = pd.read_csv('act_train.csv')
act_test = pd.read_csv('act_test.csv')
peopleData = pd.read_csv('people.csv')

#join the dataframes
trainData = act_train.merge(peopleData,on = 'people_id')


So instead of building features etc i'm going to just fill in the test data that can be accurately filled in by the leak, and then only try and predict on the 70,000 remaining points of data using a model with no group_1 predictor since the remaining test data has no training data with the same group_1 classification.

In [14]:
#convert 'dates' to dates
import datetime
mindate = (min(act_train['date']))
maxdate = (max(act_train['date']))
mindate = datetime.datetime.strptime(mindate, '%Y-%m-%d')
maxdate = datetime.datetime.strptime(maxdate, '%Y-%m-%d')
numdays = (maxdate-mindate).days

In [15]:
dates = []
for i in list(range(0,numdays)):
    dates.append(str(mindate + datetime.timedelta(days=i)).split(' ')[0])

In [16]:
elDF = [[],[]]
counter = 0
for i in list(np.unique(peopleData['group_1'].values)):
    grp = [i]*len(dates)
    appendage = [grp,dates]
    elDF[0].extend(grp)
    elDF[1].extend(dates)
    counter = counter+1

In [17]:
elDF = np.array(elDF)

In [18]:
elDF = np.transpose(elDF)
elDF = pd.DataFrame(elDF)
elDF.columns = ['group_1','date']
z = elDF.merge(trainData,how='left',left_on = ['group_1','date'],right_on = ['group_1','date_x'] )
z= z.drop_duplicates(['group_1','date'],keep ='first')
z = z[['group_1','date','outcome']]

In [19]:
#z = elDF.merge(trainData,how='left',left_on = ['group_1','date'],right_on = ['group_1','date_x'] )
#z.drop_duplicates(['group_1','date'],keep ='first')

Cool, I got all the data into a dataframe. Now I need to define an interpolation function for all the missing dates.

In [20]:
z['fwdgrp']=z['group_1'].shift(-1)
z['bwdgrp']=z['group_1'].shift(1)
z['fwddate'] = z['date'].shift(-1)
z['bwddate'] = z['date'].shift(1)
z['outcomefwd'] = z['outcome'].fillna(method='ffill')
z['outcomebwd'] = z['outcome'].fillna(method='bfill')
z['inGRP'] = (z['fwdgrp'] == z['bwdgrp']).astype(int)
z['matchingOutcomes'] = (z['outcomefwd'] == z['outcomebwd']).astype(int)
z = z[z['matchingOutcomes'] == 1]
z = z[z['inGRP'] == 1]

In [21]:
z['outcome'] = z['outcome'].fillna(z['outcomefwd'])
z = z[['group_1','date','outcome']]
z.to_csv('datatest.csv',sep=',')


Okay so the kernal keeps crashing, probbly because z is a gig on its own and I just dont have enough RAM for that kind of operation? I feel like it should just have to go to disk more which would only lead to a slowdown not a crash but who knows. Let me try with sqlite?

In [1]:
import pandas as pd
z = pd.read_csv('/Users/Ben/Desktop/Predicting Red Hat Business Value/leakOutput.csv')
act_test = pd.read_csv('/Users/Ben/Desktop/Predicting Red Hat Business Value/act_test.csv')
peopleData = pd.read_csv('/Users/Ben/Desktop/Predicting Red Hat Business Value/people.csv')
testData = act_test.merge(peopleData,on = 'people_id')
act_test = []
peopleData=[]
z = z[['group_1','date','outcome']]


In [22]:
#import sqlite3
#db = sqlite3.connect('RedHatDB.db')
#cursor = db.cursor()
##cursor.execute('CREATE TABLE IF NOT EXISTS leakData(group_1 TEXT,date TEXT,outcome INT)')

#cursor.executemany('INSERT INTO leakData(group_1,date,outcome) VALUES (?,?,?)',
              #list(z.values))
             


In [10]:
#cursor.execute('CREATE TABLE IF NOT EXISTS testData(people_id TEXT, activity_id TEXT, date_x TEXT, activity_category TEXT,char_1_x TEXT,char_2_x TEXT, char_3_x TEXT, char_4_x TEXT, char_5_x TEXT, char_6_x TEXT, char_7_x TEXT,char_8_x TEXT, char_9_x TEXT, char_10_x TEXT, char_1_y TEXT, group_1 TEXT, char_2_y TEXT,date_y TEXT, char_3_y TEXT, char_4_y TEXT, char_5_y TEXT, char_6_y TEXT, char_7_y TEXT, char_8_y TEXT,char_9_y TEXT, char_10_y TEXT, char_11 TEXT,char_12 TEXT, char_13 TEXT,char_14 TEXT, char_15 TEXT, char_16 TEXT, char_17 TEXT, char_18 TEXT, char_19 TEXT,char_20 TEXT, char_21 TEXT, char_22 TEXT, char_23 TEXT, char_24 TEXT, char_25 TEXT,char_26 TEXT, char_27 TEXT, char_28 TEXT, char_29 TEXT,char_30 TEXT, char_31 TEXT, char_32 TEXT, char_33 TEXT, char_34 TEXT, char_35 TEXT, char_36 TEXT, char_37 TEXT, char_38 TEXT)')

#cursor.execute('INSERT INTO testData(people_id, activity_id, date_x, activity_category ,char_1_x ,char_2_x , char_3_x , char_4_x , char_5_x , char_6_x , char_7_x ,char_8_x , char_9_x , char_10_x , char_1_y , group_1 , char_2_y ,date_y , char_3_y , char_4_y , char_5_y , char_6_y , char_7_y , char_8_y ,char_9_y , char_10_y , char_11 ,char_12 , char_13 ,char_14 , char_15 , char_16 , char_17 , char_18 , char_19 ,char_20 , char_21 , char_22 , char_23 , char_24 , char_25 ,char_26 , char_27 , char_28 , char_29 ,char_30 , char_31 , char_32 , char_33 , char_34 , char_35 , char_36 , char_37 , char_38) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',list(testData.values))


Keep getting some error, let me move on to a new notebook, split z up and see if I can manually tell it to save to disk etc.

In [43]:
z[z['matchingOutcomes'] == 1]

,group_1,date,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,...,char_37,char_38,fwdgrp,bwdgrp,fwddate,bwddate,outcomefwd,outcomebwd,inGRP,matchingOutcomes
133,group 1,2022-11-27,ppl_272792,act2_4346947,2022-11-27,type 2,NaN,NaN,NaN,NaN,...,False,55.0,group 1,group 1,2022-11-27,2022-11-26,0.0,0.0,1,1
134,group 1,2022-11-27,ppl_272792,act2_930969,2022-11-27,type 2,NaN,NaN,NaN,NaN,...,False,55.0,group 1,group 1,2022-11-28,2022-11-27,0.0,0.0,1,1
135,group 1,2022-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-11-29,2022-11-27,0.0,0.0,1,1
136,group 1,2022-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-11-30,2022-11-28,0.0,0.0,1,1
137,group 1,2022-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-12-01,2022-11-29,0.0,0.0,1,1
138,group 1,2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-12-02,2022-11-30,0.0,0.0,1,1
139,group 1,2022-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-12-03,2022-12-01,0.0,0.0,1,1
140,group 1,2022-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-12-04,2022-12-02,0.0,0.0,1,1
141,group 1,2022-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-12-05,2022-12-03,0.0,0.0,1,1
142,group 1,2022-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,group 1,group 1,2022-12-06,2022-12-04,0.0,0.0,1,1


9288953